In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import inv

df = pd.read_csv("/workspaces/studypython/src/ゼミ/data/calcium_full.csv")
# 目的変数の観測値
y_data = np.array(df["label"])
y_data = np.array([y_data]).T
# 説明変数の観測値ベクトルX
df_x = df.iloc[:, 1:]
one_vec = pd.Series(
    [1 for i in range(df.shape[0])], name="1"
)  # すべて1の列ベクトル（今は行かな？）
X = np.array(pd.concat([one_vec, df_x], axis=1))  # くっつけた
# フィッシャースコア法
# betaの初期化=0
beta = np.zeros(X.shape[1])
beta = np.array([beta]).T
n = df.shape[0]  # データ数n
difference = np.array([1])  # 仮の差=1
counter = 1
try:
    while difference.all() > 1e-10:
        counter += 1
        # パイ行列
        Pi = np.diag(
            np.array(
                [
                    np.exp(beta.T @ np.array([X[i]]).T)
                    / (1 + np.exp(beta.T @ np.array([X[i]]).T))
                    for i in range(X.shape[0])
                ]
            ).T[0][0]
        )
        new_beta = (
            inv(X.T @ Pi @ (np.identity(n) - Pi) @ X)
            @ X.T
            @ Pi
            @ (np.identity(n) - Pi)
            @ (
                X @ beta
                + inv(Pi @ (np.identity(n) - Pi))
                @ (y_data - Pi @ np.array([one_vec]).T)
            )
        )
        difference = new_beta - beta
        beta = new_beta

except np.linalg.LinAlgError:
    print(f"{counter}回目のループで逆行列が計算できません")

print(f"{counter}loops")
print(f"difference is {difference.max()}")
print(f"beta is {beta}")

29569loops
difference is 8.247980076703243e-10
beta is [[-3.55337709e+02]
 [ 3.55943796e+02]
 [-4.95701876e-01]
 [ 1.68112788e-02]
 [-4.32818908e-01]
 [-3.20131535e-02]
 [ 7.83691287e-01]]


In [2]:
df_x

,sg,ph,mOsm,conductivity,urea,CALC
0,1.017,5.74,577,20.0,296,4.49
1,1.008,7.20,321,14.9,101,2.36
2,1.011,5.51,408,12.6,224,2.15
3,1.005,6.52,187,7.5,91,1.16
4,1.020,5.27,668,25.3,252,3.34
...,...,...,...,...,...,...
72,1.025,7.90,721,23.6,301,9.04
73,1.017,4.81,410,13.3,195,0.58
74,1.024,5.40,803,21.8,394,7.82
75,1.016,6.81,594,21.4,255,12.20


In [3]:
import numpy as np
import pandas as pd
from numpy.linalg import inv

df = pd.read_csv("/workspaces/studypython/src/ゼミ/data/calcium_full.csv")
# 目的変数の観測値
y_data = np.array(df["label"], dtype=np.float64).reshape((df.shape[0], 1))
# 説明変数の観測値ベクトルX
df_x = df.iloc[:, 1:]
one_vec = pd.Series(
    [1 for i in range(df.shape[0])], name="1"
)  # すべて1の列ベクトル（今は行かな？）
X = np.array(pd.concat([one_vec, df_x], axis=1))

In [4]:
import itertools

factor_list = df_x.columns.values

In [5]:
def select_factor_X(df, factor_list):
    return df[factor_list]

In [6]:
from numba import njit
from numba import prange

In [7]:
# @njit()
def fisher(X, y_data):
    beta = np.zeros((X.shape[1], 1), dtype=np.float64)
    n = X.shape[0]  # データ数n
    one_vec_np = np.ones((n, 1), dtype=np.float64)
    difference = np.ones((1, 1))  # 仮の差=1

    while np.any(difference >= 1e-5):
        # Pi行列の初期化(jitで動かすためにforで回す)
        Pi = np.zeros((n, n))

        for i in range(n):
            Pi[i][i] = (np.exp(beta.T @ X[i].T) / (1 + np.exp(beta.T @ X[i].T)))[0]

        new_beta = (
            inv(X.T @ Pi @ (np.identity(n) - Pi) @ X)
            @ X.T
            @ Pi
            @ (np.identity(n) - Pi)
            @ (X @ beta + inv(Pi @ (np.identity(n) - Pi)) @ (y_data - Pi @ one_vec_np))
        )
        difference = np.abs(new_beta - beta)
        beta = new_beta

    return beta

In [ ]:
fisher(X, y_data)

In [8]:
# 空のAICデータフレーム
df_AIC = pd.DataFrame(columns=["AIC"])
df_AIC

,AIC


In [9]:
for i in range(1, len(factor_list) + 1):
    # factorからi個選ぶよん
    for selected_factor in itertools.combinations(factor_list, i):
        selected_factor = list(selected_factor)
        X = select_factor_X(df_x, selected_factor)
        X = np.array(pd.concat([one_vec, X], axis=1))
        vec_beta = fisher(X, y_data)
        AIC = (
            -2 * y_data.T @ (vec_beta.T @ X.T).reshape((X.shape[0], 1))
            + 2 * (np.log(1 + np.exp(vec_beta.T @ X.T))).sum()
            + 2 * (X.shape[1])
        )[0][0]
        AIC = pd.DataFrame(AIC, index=[",".join(selected_factor)], columns=["AIC"])
        df_AIC = pd.concat([df_AIC, AIC])
        # df_AIC = pd.concat([df_AIC, AIC])


/tmp/ipykernel_1795/413640081.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_AIC = pd.concat([df_AIC, AIC])


In [ ]:
pd.set_option("display.max_rows", None)

In [17]:
df_AIC.sort_values("AIC")

,AIC
"sg,conductivity,urea,CALC",69.071031
"sg,mOsm,conductivity,urea,CALC",70.331345
"sg,mOsm,CALC",70.368005
"sg,ph,conductivity,urea,CALC",70.501996
"mOsm,conductivity,urea,CALC",71.224283
...,...
"ph,conductivity,urea",106.221698
"ph,mOsm,urea",107.074183
ph,107.695679
conductivity,108.962751


In [15]:
df_BIC = pd.DataFrame(columns=["BIC"])
for i in range(1, len(factor_list) + 1):
    # factorからi個選ぶよん
    for selected_factor in itertools.combinations(factor_list, i):
        selected_factor = list(selected_factor)
        X = select_factor_X(df_x, selected_factor)
        X = np.array(pd.concat([one_vec, X], axis=1))
        vec_beta = fisher(X, y_data)
        BIC = (
            -2 * y_data.T @ (vec_beta.T @ X.T).reshape((X.shape[0], 1))
            + 2 * (np.log(1 + np.exp(vec_beta.T @ X.T))).sum()
            + np.log(X.shape[0]) * (X.shape[1])
        )[0][0]
        BIC = pd.DataFrame(BIC, index=[",".join(selected_factor)], columns=["BIC"])
        df_BIC = pd.concat([df_BIC, BIC])
        # df_AIC = pd.concat([df_AIC, AIC])

/tmp/ipykernel_1795/1349109427.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_BIC = pd.concat([df_BIC, BIC])


In [18]:
df_BIC.sort_values("BIC")

,BIC
"sg,mOsm,CALC",79.743226
"sg,conductivity,urea,CALC",80.790058
"mOsm,conductivity,urea,CALC",82.943310
"sg,mOsm,urea,CALC",83.088282
"sg,ph,mOsm,CALC",83.855052
...,...
"ph,mOsm",112.666759
conductivity,113.650362
"ph,conductivity,urea",115.596920
"ph,mOsm,urea",116.449405
